# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_data = pd.read_csv("API_HW.csv") 
vac_data.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Ushuaia,-54.80,-68.30,33.80,64,75,6.93,AR
1,Alyangula,-13.85,136.42,77.00,73,0,5.82,AU
2,Hithadhoo,-0.60,73.08,84.06,66,61,3.20,MV
3,Juneau,58.30,-134.42,57.20,76,75,4.70,US
4,Atuona,-9.80,-139.03,80.04,78,16,19.13,PF


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vac_data[['Latitude', 'Longitude']]
weights = vac_data['Humidity']

figure_layout = {
     'width': '400px',
     'height': '300px',
     'border': '1px solid black',
     'padding': '1px',
     'margin': '0 auto 0 auto'
}



fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = vac_data[(vac_data['Humidity'] <= 50) & (vac_data['Max Temp'] >= 70) & (vac_data['Max Temp'] <= 81) & (vac_data['Cloudiness'] < 40)]
len(ideal_df['City'])

24

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
params = {
    "keyword": 'hotel',
    "types": 'lodging',
    "radius": 5000,
    "key": g_key}

for index, row in ideal_df.iterrows():
    
    lat=row['Latitude']
    lng=row['Longitude']
    
    params['location']=f'{lat},{lng}'
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response=requests.get(url, params=params)
    
    data_response=response.json()
    
    try:
        ideal_df.loc[index, 'Hotel Name']=data_response['results'][0]['name']
    except (KeyError, IndexError):
        print('Hotel skipped')
        

ideal_df.head()


C:\Users\winyi\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\winyi\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel skipped
Hotel skipped
Hotel skipped
Hotel skipped
Hotel skipped
Hotel skipped
Hotel skipped


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
9,Yellowknife,62.46,-114.35,73.99,35,20,10.29,CA,The Explorer Hotel
78,Baza,37.49,-2.77,72.00,42,0,4.74,ES,Cuevas Al Jatib
109,Abu Kamal,34.45,40.92,79.52,38,0,15.12,SY,NaN
148,Aklavik,68.22,-135.01,73.40,35,20,5.82,CA,Sundog Inn
174,Sakakah,29.97,40.21,80.60,26,0,8.05,SA,Raoum Inn Hotel


In [7]:
#drop NaN values

ideal_df.dropna(subset=['Hotel Name'], how = 'all', inplace = True)
ideal_df.head()

C:\Users\winyi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
9,Yellowknife,62.46,-114.35,73.99,35,20,10.29,CA,The Explorer Hotel
78,Baza,37.49,-2.77,72.00,42,0,4.74,ES,Cuevas Al Jatib
148,Aklavik,68.22,-135.01,73.40,35,20,5.82,CA,Sundog Inn
174,Sakakah,29.97,40.21,80.60,26,0,8.05,SA,Raoum Inn Hotel
213,Touho,-20.78,165.23,71.73,50,0,6.71,NC,Auberge Timay Evasion


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))